In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
import numpy as np
from PIL import Image
import requests
from transformers import AutoModelForCausalLM, AutoTokenizer
from pathlib import Path
import json

from torchvision.transforms.functional import to_pil_image, to_tensor

from urllib.parse import urlparse
import requests

from train_scripts.cocodata import COCOInstancesDataset
from llavacaption import Captioner, add_descriptions

In [ ]:
loadedmodels = None
dataset = None

In [ ]:
if loadedmodels is None:
    loadedmodels = Captioner(mistralmodel=None).get_loadedmodels()
captioner = Captioner(loadedmodels=loadedmodels)

In [ ]:
if dataset is None:
    dataset = COCOInstancesDataset(maindir="/USERSPACE/lukovdg1/coco2017", split="val", upscale_to=512, max_masks=21)


In [ ]:
eids = [78,79,80] #+[500, 501, 502] #+ [1010, 1011, 1012, 1013] #+ [1005, 1006, 1007, 1008] #+ [1002, 1004, 0, 100, 1001, 50]
for eid in eids:
    example = dataset.examples[eid]
    print(example.id, example.captions)
    img = example.load_image()
    display(img)
    print("adding descriptions")
    example = add_descriptions(example, captioner)
    print("added descriptions")
    for seg in example.seg_info:
        x, y, xx, yy = seg["bbox"]
        cropped = img.crop([x, y, x + xx, y + yy])
        print(seg["category_name"], ", ", seg["description"])
        display(cropped)
        # print(seg["category_name"], (xx * yy), (xx, yy))